In [1]:
import bs4 as bs
import urllib.request
import re
import nltk
import pandas as pd

In [3]:
! pip install lxml

   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.8 MB 1.3 MB/s eta 0:00:03
    --------------------------------------- 0.1/3.8 MB 653.6 kB/s eta 0:00:06
   - -------------------------------------- 0.2/3.8 MB 1.1 MB/s eta 0:00:04
   -- ------------------------------------- 0.2/3.8 MB 1.4 MB/s eta 0:00:03
   ---- ----------------------------------- 0.4/3.8 MB 1.7 MB/s eta 0:00:02
   ------ --------------------------------- 0.6/3.8 MB 2.2 MB/s eta 0:00:02
   -------- ------------------------------- 0.8/3.8 MB 2.4 MB/s eta 0:00:02
   ---------- ----------------------------- 1.0/3.8 MB 2.5 MB/s eta 0:00:02
   ------------ --------------------------- 1.2/3.8 MB 2.7 MB/s eta 0:00:01
   -------------- ------------------------- 1.4/3.8 MB 2.8 MB/s eta 0:00:01
   ---------------- ----------------------- 1.6/3.8 MB 2.9 MB/s eta 0:00:01
   ----------------- ---------------------- 1.6/3.8 MB 3.0 MB/s eta 0:00:01
   --------------

In [7]:
!pip install beautifulsoup4

In [9]:
pip install beautifulsoup4 lxml

Note: you may need to restart the kernel to use updated packages.


In [2]:
data = urllib.request.urlopen('https://en.wikipedia.org/wiki/Artificial_intelligence')
article = data.read()
parsed = bs.BeautifulSoup(article,'lxml')
ps = parsed.find_all('p')
txt = ""
for p in ps:
    txt += p.text

In [5]:
txt[:1000]

'\nArtificial intelligence (AI), in its broadest sense, is intelligence exhibited by machines, particularly computer systems. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and uses learning and intelligence to take actions that maximize their chances of achieving defined goals.[1] Such machines may be called AIs.\nAI technology is widely used throughout industry, government, and science. Some high-profile applications include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); interacting via human speech (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., ChatGPT and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go).[2] However, many AI applications are not perceived as AI: "A lot of cutting edge AI has filtered into general applicatio

In [6]:
txt = re.sub(r'\[[0-9]*\]',' ',txt)
txt = re.sub(r'\s+',' ',txt)
text = re.sub(r'[^A-Za-z]',' ',txt)
text = re.sub(r'\s+',' ',text)

In [7]:
text[:500]

' Artificial intelligence AI in its broadest sense is intelligence exhibited by machines particularly computer systems It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and uses learning and intelligence to take actions that maximize their chances of achieving defined goals Such machines may be called AIs AI technology is widely used throughout industry government and science Some high profile applicatio'

In [8]:
sents = nltk.sent_tokenize(txt)
stopwords = nltk.corpus.stopwords.words("english")
word_freq = {}
for w in nltk.word_tokenize(text):
    if w not in stopwords:
        w = w.lower()
        if w not in word_freq.keys():
            word_freq[w] = 1
        else:
            word_freq[w] += 1

In [9]:
mx = max(word_freq.values())
for w in word_freq.keys():
    word_freq[w] = word_freq[w] / mx

In [10]:
pd.Series(word_freq)[:10]

artificial      0.215789
intelligence    0.321053
ai              1.000000
broadest        0.005263
sense           0.026316
exhibited       0.010526
machines        0.078947
particularly    0.026316
computer        0.084211
systems         0.105263
dtype: float64

In [11]:
for s in sents[:5]:
    print(s)
    print("-"*50)

 Artificial intelligence (AI), in its broadest sense, is intelligence exhibited by machines, particularly computer systems.
--------------------------------------------------
It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and uses learning and intelligence to take actions that maximize their chances of achieving defined goals.
--------------------------------------------------
Such machines may be called AIs.
--------------------------------------------------
AI technology is widely used throughout industry, government, and science.
--------------------------------------------------
Some high-profile applications include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); interacting via human speech (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., ChatGPT and AI a

In [12]:
sen_scores = {}
for s in sents:
    for w in nltk.word_tokenize(s):
        w = w.lower()
        if w in word_freq.keys():
            if len(s.split(' ')) < 30:
                if s not in sen_scores.keys():
                    sen_scores[s] = word_freq[w]
                else:
                    sen_scores[s] += word_freq[w]

In [13]:
sen_scores

{' Artificial intelligence (AI), in its broadest sense, is intelligence exhibited by machines, particularly computer systems.': 2.478947368421052,
 'Such machines may be called AIs.': 0.3105263157894737,
 'AI technology is widely used throughout industry, government, and science.': 1.4842105263157896,
 'Alan Turing was the first person to conduct substantial research in the field that he called machine intelligence.': 2.0684210526315794,
 'Artificial intelligence was founded as an academic discipline in 1956.': 0.8789473684210525,
 'The field went through multiple cycles of optimism, followed by periods of disappointment and loss of funding, known as AI winter.': 1.6157894736842107,
 'Funding and interest vastly increased after 2012 when deep learning surpassed all previous AI techniques, and after 2017 with the transformer architecture.': 1.973684210526316,
 'This led to the AI boom of the early 2020s, with companies, universities, and laboratories overwhelmingly based in the United S

In [16]:
import heapq
summary_sentences = heapq.nlargest(7, sen_scores, key=sen_scores.get)

summary = '\n'.join(summary_sentences)
print(summary)

In May 2024 at the AI Seoul Summit, 16 global AI tech companies agreed to safety commitments on the development of AI.
This led to the AI boom of the early 2020s, with companies, universities, and laboratories overwhelmingly based in the United States pioneering significant advances in artificial intelligence.
In-production systems can sometimes not factor ethics and bias into their AI training processes, especially when the AI algorithms are inherently unexplainable in deep learning.
K-nearest neighbor algorithm was the most widely used analogical AI until the mid-1990s, and Kernel methods such as the support vector machine (SVM) displaced k-nearest neighbor in the 1990s.
AI pioneer Juergen Schmidhuber did not sign the joint statement, emphasising that in 95% of all cases, AI research is about making "human lives longer and healthier and easier."
The AI learned that users tended to choose misinformation, conspiracy theories, and extreme partisan content, and, to keep them watching, th